du 13/03/2023 au 22/03/2023
notebook 1
objectif : premier essai du système 
contexte : travail individuel 
difficulté : je ne parviens pas à enregistrer les résultats au format JSON à cause du format particulier "EntityCollection".

# Test du sytème d'entity linking de Spacy

## Source

J'ai repris le code suivant proposé sur  : https://github.com/egerber/spaCy-entity-linker

In [1]:
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

doc = nlp("I watched the Pirates of the Caribbean last silvester")

# returns all entities in the whole document
all_linked_entities = doc._.linkedEntities
# iterates over sentences and prints linked entities
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.

##  J'ai adapté le code pour qu'il prenne en entrée un fichier JSON


Je l'ai adapté pour qu'il prenne en entrée le fichier JSON contenant des listes d'entités nommées. J'ai également changé le modèle de langue pour le mettre en français. 

In [ ]:
import json
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
newdata = json.dumps(data)
doc = nlp(newdata)


#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
#iterates over sentences and prints linked entities
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()

## Les mots dans leur contexte

J'ai pensé que peut-être les erreurs étaient liées au manque de contexte. J'ai donc cherché à voir si le résultat est différent si je donnais en entrée le texte océrérisé. 

In [2]:
import json
import spacy
from spacy_entity_linker import EntityLinker

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# initialize entity linker
nlp.add_pipe("entityLinker", last=True)

with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt','r', encoding='UTF-8') as f:
    data = f.read()
doc = nlp(data)


# print linked entities for each sentence
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()


<EntityElement: https://www.wikidata.org/wiki/Q21450529 Duclos                    family name                                       >
<EntityElement: https://www.wikidata.org/wiki/Q69546 Le Lieu                   municipality in Switzerland                       >
<EntityElement: https://www.wikidata.org/wiki/Q14953 Lage                      town in North Rhine-Westphalia, Germany           >
<EntityElement: https://www.wikidata.org/wiki/Q870 train                     one or more self-propelled vehicles that move along fixed rails to transport passengers, animals or >
<EntityElement: https://www.wikidata.org/wiki/Q560 helium                    chemical element with the chemical symbol He and the atomic number of 2>
<EntityElement: https://www.wikidata.org/wiki/Q7813319 Toi                       name                                              >
<EntityElement: https://www.wikidata.org/wiki/Q130964 calorie                   unit of energy                                    >
<EntityEle

En donnant le texte en entier, on remarque que les entitées reconnues ne sont pas les mêmes, il semble donc intéressant de pouvoir croiser ces deux sources. Ensuite, on remarque  que parmis les éléments reconnus, beaucoup sont des termes homonymes en français et anglais mais qui n'ont pas le même sens dans les deux langues comme "coin". Le modèle de langue a beau être defini en français, le mot "coin" est reconnu dans son sens anglais. De plus, on peut observer que les définitions données par wikidata peuvent parfois sembler surprenantes ; en particulier, celle du mot femme : "feminine lesbian or bisexual woman". Le mot "femme" est en effet employé en anglais dans ce sens.  L'algorithme semble donc privilégier l'anglais sans prendre en compte les particularités des autres langues. 

https://www.kaggle.com/datasets/kenshoresearch/kensho-derived-wikimedia-data/discussion/342001

Le créateur du dataset, Gabriel Altay, a répondu ainsi à une question sur le multilinguisme : "Wikidata (the knowledge graph) is semi language agnostic. In other words, there is one entry for "Earth" that links to the various language Wikipedias (encyclopedias). This dataset links English Wikipedia page text and links to Wikidata. Something similar could be done for Chinese Wikipedia, but it has not been done for this dataset." 

De plus, il semble que l'algorithme ait recours à une correction automatique. En effet, les termes reconnus souvent ne figurent pas dans le corpus même sous un forme bruitée : "Aurora". L'algorithme a sûrement fait le rapprochement en corrigeant une forme contaminée. En recherchant dans le corpus, on ne parvient pas à trouver la forme à l'origine de cette surcorrection. 

## Enregistrer les sorties dans un fichier JSON. 

### Premier essai : 

In [3]:
import json
import spacy

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

# open the input file and load its content as a dictionary
with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)

newdata = json.dumps(data)
# process the data with the NER model
doc = nlp(newdata)


# get all linked entities in the whole document
all_linked_entities = doc._.linkedEntities

linked_entities_by_sent = []
for sent in doc.sents:
    sent_linked_entities = sent._.linkedEntities.to_json()
    linked_entities_by_sent.append(str(sent_linked_entities))
print(type(sent_linked_entities))

# write the linked entities to a JSON file
with open('output_file.json', 'w', encoding='utf-8') as f:
    json.dump(linked_entities_by_sent, f, ensure_ascii=False, indent=4)


AttributeError: 'EntityCollection' object has no attribute 'to_json'

### premier essai avec  correction de Caroline Parfait

In [ ]:
import json
import spacy

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

# open the input file and load its content as a dictionary
with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)

newdata = json.dumps(data)
# process the data with the NER model
doc = nlp(newdata)


# get all linked entities in the whole document
all_linked_entities = doc._.linkedEntities


#List<myEntity> entityList = new List<myEntity>(myEntityCollection);
print(type(sent_linked_entities))

# create a list of linked entities for each sentence
linked_entities_by_sent = []
for sent in doc.sents:
    sent_linked_entities = sent._.linkedEntities.to_json()
    #sent_linked_entities = sent._.linkedEntities.to_json()
    linked_entities_by_sent.append(str(sent_linked_entities))

# write the linked entities to a JSON file
with open('output_file.json', 'w', encoding='utf-8') as f:
    json.dump(linked_entities_by_sent, f, ensure_ascii=False, indent=4)


### Deuxième essai : en registrant dans un dictionnaire

Visiblement, ce code ne fonctionne pas. J'ai donc essayé avec celui-ci. Cette fois-ci, j'ai essayé d'enregistrer les entités dans un dictionnaire.

In [ ]:
import json
from spacy.tokens import Doc
import spacy

nlp = spacy.load("fr_core_news_md")
nlp.add_pipe("entityLinker", last=True)

with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
# charger les données
    data = json.load(f)

doc = Doc(nlp.vocab).from_dict(data)

# extraire les entités liées
linked_entities = doc._.linkedEntities

print(type(linked_entities))

# créer une liste des entités
entities_list = [entity.to_dict() for entity in linked_entities]

# enregistrer les entités dans un fichier JSON
with open("fichier_entites.json", "w", encoding="utf-8") as f:
    json.dump(entities_list, f, ensure_ascii=False, indent=4)

### Troisième essai

et comme ça ne fonctionne toujours pas, j'ai aussi essayé celui-là.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 13 11:50:05 2023

@author: dorle
"""
import json
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("fr_core_news_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)
with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
doc = nlp(json.dumps(data))

#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
#iterates over sentences and prints linked entities
with open("entite.json","w") as f:
    for sent in doc.sents:
        json.dump((all_linked_entities),f)
        sent._.linkedEntities.pretty_print()
    

### quatrième essai : en enregistrant dans un tableau

Comme toutes les erreurs des codes précédents semblent liées au EntityCollection, j'ai cherché à contourner le problème en convertissant l'EntityCollection en tableau 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 13 11:50:05 2023

@author: dorle
"""
import json
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("fr_core_news_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)
with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
doc = nlp(json.dumps(data))


#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
#iterates over sentences and prints linked entities

entities = []

for entity in doc._.linkedEntities:
    entity_dict = {
        "text": entity.text,
        "label": entity.label_,
        "kb_id": entity.kb_id_
    }
    entities.append(entity_dict)
    
print(entities)

### Affichage 

Pour résoudre la problème, Caroline m'a conseillé d'afficher le type de la variable

In [ ]:
import json
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)
with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
newdata = json.dumps(data)
print(newdata)
print(type(newdata))

doc = nlp(newdata)


#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
print(type(all_linked_entities ))
print(all_linked_entities)
#iterates over sentences and prints linked entities



"EntityCollection contains an array of entity elements. It can be accessed like an array but also implements the following helper functions:

pretty_print() prints out information about all contained entities
print_super_classes() groups and prints all entites by their super class"

## Méthode de Caroline Parfait pour charger le JSON

Caroline Parfait m'a suggéré de prendre exemple sur le code cluster.py

In [ ]:
import json
import spacy
import glob

def lire_fichier (chemin):
    with open(chemin) as json_data: 
        texte =json.load(json_data)
    return texte

def stocker(chemin,contenu):                                                    #definition fonction pour stocker fichier en format json
    w=open(chemin,"w")                                                          #ouverture du fichier en mode écriture
    w.write(json.dumps(contenu, indent=2))                                      #écriture du contenu dans le fichier
    w.close()  

path_corpora = "../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json"
newdata = lire_fichier(path_corpora)
print(type(newdata))



## Cinquième essai : En enregistrant en CSV

In [ ]:
import json
import spacy  # version 3.5
import pandas as pd
import csv

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
newdata = json.dumps(data)
doc = nlp(newdata)


#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
#iterates over sentences and prints linked entities


with open('entities.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(all_linked_entities)


In [4]:
import json
import spacy  # version 3.5

# initialize language model
nlp = spacy.load("fr_core_news_lg")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

with open('../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/ADAM_krakenbase/ADAM_MOD/ADAM_Mon-village_Kraken-base.txt_spacy-lg.json','r', encoding='UTF-8') as f:
    data = json.load(f)
newdata = json.dumps(data)
doc = nlp(newdata)


#returns all entities in the whole document

all_linked_entities = doc._.linkedEntities
#iterates over sentences and prints linked entities

label = all_linked_entities.get_label()
print (label)



AttributeError: 'EntityCollection' object has no attribute 'get_label'

# Comparer les sorties 

Une fois les sorties définies, il semble intérressant de pouvoir comparer les sorties des deux programmes : les entitités dans leur contexte et celles hors de leur contexte. Afin de déterminer, s'il existe une différence. 

In [ ]:
import json

# Charger le contenu du premier fichier JSON
with open("chemin.json", "r") as f:
    data1 = json.load(f)

# Charger le contenu du deuxième fichier JSON
with open("chemin.json", "r") as f:
    data2 = json.load(f)

# Extraire les valeurs de chaque fichier
values1 = set(data1.values())
values2 = set(data2.values())

# Trouver les valeurs communes
common_values = values1.intersection(values2)

# Afficher les valeurs communes
print("Les valeurs communes sont :", common_values)